## Hierarchical Architecture

In [57]:
from dotenv import load_dotenv
load_dotenv()

True

In [58]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain.chat_models import ChatOpenAI

from langgraph.graph import END, StateGraph, MessageGraph
from langgraph.checkpoint.sqlite import SqliteSaver

import functools
import operator
from typing import List, Sequence, TypedDict, Annotated
import json
import os
import random

from IPython.display import Image, display

import concurrent.futures

In [59]:
unique_id = "Hierarchical Optimisation"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"

In [60]:
# from langsmith import Client

# client = Client()

In [61]:
class PromptReview(BaseModel):
    """Review of the prompt"""
    feedback: str = Field(description="Feedback on the most recent prompt")

class CorePrinciples:
    def __init__(self, core_principles: List[str]):
        self.core_principles = core_principles
    
    def add_principle(self, principle: str):
        """
        Adds a principle to the core principles list.
        
        :param principle: The principle to be added.
        """
        self.core_principles.append(principle)
        
    def __str__(self):
        """
        Returns a string representation of the core principles, each principle is listed on a new line with a preceding dash.
        
        Example:
        - principle 1
        - principle 2
        ...
        """
        return "\n".join([f"- {principle}" for principle in self.core_principles])

class WorkerAgent:
    """
    Worker Agent class defining agents that provide feedback on prompts.
    """

    def __init__(self, position: str, core_principles: CorePrinciples, temp: float = 1.0, model: str = "gpt-4o"):
        self.position = position
        self.core_principles = str(core_principles)
        self.system_message = SystemMessage(content=f"""You are an experienced: {self.position}. Your core principles are:
{self.core_principles}""")        
        self.llm = ChatOpenAI(
            temperature=temp,
            model=model,
        )

    def review_prompt(self, prompt: str, criteria: str) -> PromptReview:
        """
        Generates a review of the prompt.
        """
        template = """```{prompt}```

Your task is to think outside the box and provide feedback on the prompt above with creative recommendations on how to improve it in light of your core principles.

The success criteria for the updated prompt are as follows:
{criteria}
Use this information to inform your feedback and recommendations.

Your feedback must be actionable and less than 100 words. 
Provide clear and concise instructions on how to implement your recommendations.

Below are strict guidelines that you MUST follow when providing feedback and recommendations:
- DO NOT suggest modifying existing restrictions.
- DO NOT suggest modifying or removing negations.
- DO NOT suggest adding, modifying or removing placeholders denoted by curly braces. IT IS ESSENTIAL PLACEHOLDERS REMAIN UNCHANGED.
- ALWAYS treat placeholders as the actual content.
You will be penalized is you do not follow these guidelines.

Your reviewal process should be as follows:
1. Read the prompt carefully as an expert {position}. 
2. Identify the most critical aspects of the prompt that need improvement. 
3. Think outside the box to provide creative feedback with recommendations on how to improve the prompt in light of your core principles.
4. Ensure that your feedback is less than 100 words.
5. Ensure that your feedback follows the strict guidelines provided above.
6. Submit your feedback.

{format_instructions}

Output only your feedback. Example output:

{{
    "feedback": "Feedback on the original prompt"
}}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=PromptReview)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["position", "prompt", "criteria"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                completion = chain.invoke({"position": self.position, "prompt": prompt, "criteria": criteria})
                # Validate the output before returning
                if completion.feedback:
                    return completion
                # else:
                #     print("Validation failed: Missing required fields in completion")
                #     print("Raw output:", completion)
            except Exception as e:
                print("Exception occurred:", e)
                continue
        else:
            raise Exception("Failed to parse output after 3 attempts")

In [62]:
class Workforce(BaseModel):
    """Details of workforce generated by the leader agent."""
    positions: List[str] = Field(description="Positions of the workers in the workforce")
    core_principles: List[List[str]] = Field(description="Core principles of the workers in the workforce")
    

class FeedbackSummary(BaseModel):
    """Summary of feedback from the worker agents."""
    feedback_summary: str = Field(description="Collated and summarised feedback from the workforce")


class ApprovalDecision(BaseModel):
    """Decision of the leader agent to approve or disapprove the prompt."""
    approved: bool = Field(description="Decision to approve or disapprove the prompt")


class TeamLeaderAgent:
    """
    TeamLeaderAgent class defining an agent that manages a team of worker agents to help optimise prompts.
    """

    def __init__(self, team: str, core_principles: CorePrinciples, prompt: str, criteria: str = None, temp: float = 0.5, model: str = "gpt-4o", workforce: List[WorkerAgent] = None):
        self.position = team
        self.core_principles = str(core_principles)
        self.system_message = SystemMessage(content=f"""You are an experienced: {self.position}. Your core principles are:
{self.core_principles}""")
        self.prompt = prompt
        self.criteria = criteria
        self.llm = ChatOpenAI(
            temperature=temp,
            model=model,
        )
        self.workforce = workforce

    def leader_feedback(self, prompt: str, feedback: list) -> str:
        """
        LeaderAgent to decide the next worker or to finish.
        """
        # All workers except the current worker
        template = """Your task is to refine and summarise the feedback below.    
```{feedback}```


You must capture the important aspects of the feedback and recommendations provided by your team.
Your summary must be actionable and precise, providing clear recommendations for improvements.

The success criteria for the updated prompt are as follows:
{criteria}
Use this information to inform your decision.

Below are strict guidelines that you MUST follow when summarising feedback and recommendations:
- DO NOT suggest modifying existing restrictions.
- DO NOT suggest modifying or removing negations.
- DO NOT suggest adding, modifying or removing placeholders denoted by curly braces. IT IS ESSENTIAL PLACEHOLDERS REMAIN UNCHANGED.
- ALWAYS treat placeholders as the actual content.
You will be penalized if you do not follow these guidelines.

Your summarisation process should be as follows:
1. Read the feedback carefully as an experienced {position}.
2. Identify the important aspects of the feedback and recommendations provided by your team.
3. Refine and summarise the feedback and recommendations into a concise and actionable summary.
4. Ensure that your summary adheres to the strict guidelines provided above.
5. Ensure that your summary is actionable and precise, providing clear recommendations for improvements to the prompt.
6. Submit your summary.

{format_instructions}

Output only the feedback summary. Example output:

{{
    "feedback_summary": "Feedback summary",
}}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=FeedbackSummary)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["feedback", "prompt", "criteria", "position"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                output = chain.invoke({"feedback": feedback, "prompt": prompt, "criteria": self.criteria, "position": self.position})
                break
            except Exception as e:
                print("Exception occurred:", e)
                continue
        return output.feedback_summary
       
    def get_feedback(self, state):
        """
        Get feedback from wokers. Feedback collected concurrently.
        """
        with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
            futures = [executor.submit(worker.review_prompt, state['prompt'], self.criteria) for worker in self.workforce]
            results = [future.result() for future in concurrent.futures.as_completed(futures)]
            # feedback = [result[-1].content for result in results]
            feedback = [result.feedback for result in results]

        summary = self.leader_feedback(state["prompt"], feedback)

        return summary
    
    def get_approval(self, prompt: str):
        """
        Agent to approve or reject the prompt.
        """
        template = """```{prompt}```

Your task is to review the prompt above and decide whether or not it should be approved in light of your team and team role.

Return True if you approve and think the prompt already optimal in the aspects your team:, {members} specialises in. 
Return False if you disapprove and think the prompt could be improved with the insights of your team.

The success criteria for the updated prompt are as follows:
{criteria}
Use this information to inform your feedback.

Your reviewal process should be as follows:
1. Read the prompt carefully as an expert {position}
2. Determine whether the prompt will be effective in instructing the model to perform the desired task.
3. If you believe the prompt is effective, approve it for use by the model.
4. Submit your decision.
    
{format_instructions}

Output only your approval decision. Example output:

{{
    "approved": "Approval decision"
}}

You will be penalized if your output cannot be parsed correctly.
"""
        pydantic_parser = PydanticOutputParser(pydantic_object=ApprovalDecision)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["prompt", "members", "criteria", "position"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                completion = chain.invoke({"prompt": prompt, "members": [worker.position for worker in self.workforce], "criteria": self.criteria, "position": self.position})
                # Validate the output before returning
                if completion.approved is not None:
                    return completion.approved
                # else:
                #     print("Validation failed: Missing required fields in completion")
                #     print("Raw output:", completion)
            except Exception as e:
                print("Exception occurred:", e)
                continue
        else:
            raise Exception("Failed to parse output after 3 attempts")

In [63]:
class Domain(BaseModel):
    """Domain of the prompt."""
    domain: str = Field(description="Domain of the prompt")
    description: str = Field(description="Description of the domain team role")


class RouteDecision(BaseModel):
    """Decision on the next worker to process."""
    next: str = Field(description="The next team to process")


class UpdatedPrompt(BaseModel):
    """Updated prompt based on feedback from the team leader."""
    updated_prompt: str = Field(description="Updated prompt based on feedback")


class LeaderAgent:
    """
    LeaderAgent class defining an agent that generates and communicates worker agents to help optimise prompts.
    """

    def __init__(self, base_prompt: str, criteria: str = None, temp: float = 0.0, model: str = "gpt-4o", team_leaders: List[TeamLeaderAgent] = None):
        self.system_message = SystemMessage(content=f"""You are an experienced: Head AI Engineer. Your core principles are:
- Always adhere to strict guidelines
- Always strive to achieve success criteria
- Always listen to feedback
- Always consider the limitations of AI models""")        
        self.base_prompt = base_prompt
        self.prompt = base_prompt
        self.criteria = criteria
        self.llm = ChatOpenAI(
            temperature=temp,
            model=model,
        )
        self.team_leaders = team_leaders
        self.iterations = 0
    
    def run_approval(self, prompt: str) -> List[bool]:
        """
        Run the approval process for the prompt. Run concurrently
        """
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = [executor.submit(team_leader.get_approval, prompt) for team_leader in self.team_leaders]
            results = [future.result() for future in concurrent.futures.as_completed(futures)]
        return results
    
    def leader_decision(self, state: dict) -> RouteDecision:
        """
        LeaderAgent to decide the next worker or to finish.
        """
        self.iterations += 1
        approval_results = self.run_approval(state["prompt"])
        print("Approval results:", approval_results)
        if all(approval_results) or (self.iterations > 10):
            return {"next": "FINISH", "prompt": state["prompt"], "messages": state["messages"]} 
        else:
            self.iterations += 1
            disapproved_team_leaders = [team_leader for i, team_leader in enumerate(self.team_leaders) if not approval_results[i]]
            options = [team_leaders.position for team_leaders in disapproved_team_leaders]
            # shuffle options to avoid positional bias
            random.shuffle(options)
            # options = ["FINISH"] + members
            template = """```{prompt}```

Your task is to review the prompt above and decide the next team to provide feedback.

The success criteria for the updated prompt are as follows:
{criteria}
Use this information to inform your decision.

Select one of the following teams to provide feedback on the prompt: 
{options}

Think carefully about pairing the aspects of the prompt that need improvement and how they relate to the expertise of each team.
If you think multiple aspects of the prompt need improvement, select the most suitable team to provide feedback on the most critical aspects of the prompt.

{format_instructions}

Output only the next team name to process. Example output:

{{
    "next": "Next team",
}}

You will be penalized if your output cannot be parsed correctly."""
            pydantic_parser = PydanticOutputParser(pydantic_object=RouteDecision)
            prompt_template = PromptTemplate(
                system_message=self.system_message,
                template=template,
                input_variables=["prompt", "criteria", "options"],
                partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
            )
            chain = prompt_template | self.llm | pydantic_parser
            for _ in range(3):
                try:
                    output = chain.invoke({"prompt": state["prompt"], "criteria": self.criteria, "options": options})
                    break
                except Exception as e:
                    print("Exception occurred:", e)
                    continue
            return {"next": output.next, "prompt": state["prompt"], "messages": state["messages"]}

    def update_prompt(self, prompt: str, feedback: str, history) -> str:
        """
        Updates the prompt with the feedback from the worker agent.
        """
        template = """```{prompt}```

Your task is to make updates to the prompt above by implementing the feedback below. You must action the feedback keeping in mind your core principles.
```{feedback}```

The success criteria for the updated prompt are as follows:
{criteria}
You will be penalized if the updated prompt does not meet this criteria.

Below are strict guidelines that you MUST follow if making changes to the prompt:
- DO NOT modify existing restrictions.
- DO NOT modify or remove negations.
- DO NOT add, modify or remove placeholders denoted by curly braces. IT IS ESSENTIAL PLACEHOLDERS REMAIN UNCHANGED.
- ALWAYS treat placeholders as the actual content.
You will be penalized if your do not follow these guidelines.

Your update process should be as follows:
1. Read the prompt and feedback carefully.
2. Review the discussion history to ensure you do not repeat any mistakes.
3. Implement the feedback provided to help the prompt better achieve what is expected.
4. Ensure that your update follows the strict guidelines provided above.
5. Submit your updated prompt.

{format_instructions}

Output only your updated prompt. Example output:

{{
    "updated_prompt": "Updated prompt",
}}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=UpdatedPrompt)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["prompt", "feedback", "history", "criteria"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                output = chain.invoke({"prompt": prompt, "feedback": feedback, "history": history, "criteria": self.criteria})
                if output.updated_prompt:
                    return output.updated_prompt
                # else:
                #     print("Validation failed: Missing required fields in completion")
                #     print("Raw output:", output)
            except Exception as e:
                print("Exception occurred:", e)
                continue

    def construct_team_graph(self):
        """
        Constructs a graph of team leader agents.
        """
        # The agent state is the input to each node in the graph
        class TeamState(TypedDict):
            # The annotation tells the graph that new messages will always be added to the current states
            messages: Sequence[BaseMessage]
            prompt: str
            next: str

        def team_node(state, team_leader):
            try:
                feedback = team_leader.get_feedback(state)
                updated_prompt = self.update_prompt(state["prompt"], feedback, state["messages"])
                # updated_prompt = output[-1].content 
                self.prompt = updated_prompt
                return {
                    "messages": state["messages"] + [
                        HumanMessage(content=f"Feedback: {feedback}", name=team_leader.position),
                        AIMessage(content=f"Updated Prompt: {updated_prompt}", name="Leader")
                    ],
                    "prompt": updated_prompt,
                }
            except Exception as e:
                # Log the error and return to leader with the most recent prompt
                print(f"Parsing failed for {team_leader.position}: {e}")
                return {
                    "messages": state["messages"] + [
                        HumanMessage(content=f"Error: Parsing failed for {team_leader} - {e}", name=team_leader.position),
                        AIMessage(content=f"Updated Prompt: {updated_prompt}", name="Leader")
                    ],                    
                    "prompt": state["prompt"],
                    "next": "leader",
                }

        workflow = StateGraph(TeamState)
        for team_leader in self.team_leaders:
            # Create a node for each team leader agent
            # team_leader.construct_worker_graph()
            node = functools.partial(team_node, team_leader=team_leader)
            workflow.add_node(team_leader.position, node)
        workflow.add_node("leader", self.leader_decision)

        members = [team_leader.position for team_leader in self.team_leaders]
        for member in members:
            # We want our workers to ALWAYS "report back" to the leader when done
            workflow.add_edge(member, "leader")
        # The leader populates the "next" field in the graph state with routes to a node or finishes
        conditional_map = {k: k for k in members}
        conditional_map["FINISH"] = END
        workflow.add_conditional_edges("leader", lambda x: x["next"], conditional_map)
        # Finally, add entrypoint
        workflow.set_entry_point("leader")

        memory = SqliteSaver.from_conn_string(":memory:")
        graph = workflow.compile(checkpointer=memory)
        
        return graph
    
    def optimise_prompt(self):
        """
        Optimises a prompt by invoking a graph of worker agents.
        """
        # Initial state
        initial_state = {
            "messages": [HumanMessage(content=f"{self.base_prompt}", name="User")],
            "prompt": self.base_prompt,
            "next": "leader",
        }

        # Construct the graph
        graph = self.construct_team_graph()
        # display(Image(graph.get_graph().draw_mermaid_png()))

        n = random.randint(0, 1000)
        config = {
            "configurable": {"thread_id": str(n)},
            "recursion_limit": 50,
            }    

        # Run the graph
        for s in graph.stream(
            initial_state,
            config,
            stream_mode="values",
            ):
            if "__end__" not in s:
                if len(s["messages"]) > 2:
                    s["messages"][-2].pretty_print()
                    s["messages"][-1].pretty_print()
                elif len(s["messages"]) > 1:
                    s["messages"][-1].pretty_print()
                continue

        # if not os.path.exists("prompt_history_hierarchical.json"):
        #     with open("prompt_history_hierarchical.json", "w") as f:
        #         json.dump([], f)
        
        # with open("prompt_history_hierarchical.json", "r") as f:
        #     data = json.load(f)
        #     data.append(self.prompt_history)
            
        # with open("prompt_history_hierarchical.json", "w") as f:
        #     json.dump(data, f, indent=4)
                
        return s

In [64]:
# Prompt design team members
style_and_structure_principles = CorePrinciples([
    "Always structure prompts logically, for example COSTAR (Context, Objective, Steps, Task, Additional Information, Result)",
    "Always use a style and tone in prompts that is appropriate for the task",
    "Always consider the complexity of the task when designing prompts",
])
style_and_structure_expert = WorkerAgent("Style and Structure Expert", style_and_structure_principles)

conciseness_and_clarity_principles = CorePrinciples([
    "Always write clear and concise prompts",
    "Always use simple and direct language in prompts",
    "Always consider the task criteria when formatting and structuring prompts",
])
conciseness_and_clarity_expert = WorkerAgent("Conciseness and Clarity Expert", conciseness_and_clarity_principles)

contextual_relevance_principles = CorePrinciples([
    "Always provide context to help the model understand the task",
    "Always consider the context in which prompts will be used",
    "Always ensure personas and roles in prompts are optimal to the context of the task",
])
contextual_relevance_expert = WorkerAgent("Contextual Relevance Expert", contextual_relevance_principles)

task_alignment_principles = CorePrinciples([
    "Always ensure that prompts align with the task criteria",
    "Always tailor instructions to the task to guide the model",
])
task_alignment_expert = WorkerAgent("Task Alignment Expert", task_alignment_principles)

example_demonstration_principal = CorePrinciples([
    "Always provide examples to help the model understand the task",
    "Always ensure examples are relevant and clear",
    "Always demonstrate the expected output of the model",
])
example_demonstration_expert = WorkerAgent("Example Demonstration Expert", example_demonstration_principal)

avoiding_bias_principles = CorePrinciples([
    "Always avoid bias in prompts",
    "Always consider the ethical implications of prompts",
])
avoiding_bias_expert = WorkerAgent("Avoiding Bias Expert", avoiding_bias_principles)

incremental_prompting_principles = CorePrinciples([
    "Always provide clear step-by-step instructions to guide the model",
    "Always consider the complexity of the task when providing instructions",
])
incremental_prompting_expert = WorkerAgent("Incremental Prompting Expert", incremental_prompting_principles)

programming_logic_principles = CorePrinciples([
    "Only implement programming logic if you think it is beneficial for the task",
    "Structure prompts logically, similar to programming logic",
    "Implement programming logic including loops, conditionals, and functions and even pseudo-code",
])
programming_logic_expert = WorkerAgent("Programming Logic Expert", programming_logic_principles)

In [65]:
# Domain team members
mathematics_principles = CorePrinciples([
    "Always follow good mathematical practices",
    "Always use mathetical operators correctly",
    "Always consider the mathematical principles relevant to the task",
])
mathematician = WorkerAgent("Mathematician", mathematics_principles)

word_problem_solving_principles = CorePrinciples([
    "Always pay attention to the keywords in problems",
    "Always approach problems systematically",
    "Always consider multiple approaches to solving problems",
])
word_problem_solver = WorkerAgent("Word Problem Solver", word_problem_solving_principles)


In [66]:
base_prompt = "{content}\nPlease output your answer at the end as ##<your answer (arabic numerals)>."
criteria = """- The prompt correctly instructs an LLM to solve the simple maths problem provided.
- The prompt correctly outputs the answer at the end as ##<answer> with no spaces or units."""

prompt_design_team = TeamLeaderAgent(
    "Lead Prompt Writer",
    CorePrinciples([
        "Always follow good prompt design practices",
        "Always adhere to strict guidelines",
        "Always listen to feedback if received",
    ]),
    base_prompt,
    criteria,
    workforce=[
        style_and_structure_expert,
        conciseness_and_clarity_expert,
        contextual_relevance_expert,
        task_alignment_expert,
        example_demonstration_expert,
        incremental_prompting_expert,
        programming_logic_expert,
    ]
)
domain_team = TeamLeaderAgent(
    "Lead Mathematician",
    CorePrinciples([
        "Always follow good mathematical practices",
        "Always adhere to strict guidelines",
        "Always listen to feedback if received",
    ]),
    base_prompt,
    criteria,
    workforce=[
        mathematician,
        word_problem_solver,
    ]
)

leader_agent = LeaderAgent(
    base_prompt=base_prompt,
    criteria=criteria,
    team_leaders=[prompt_design_team, domain_team]
)

# leader_agent.generate_teams()
# print teams and members
print("Teams and members:")
for team_leader in leader_agent.team_leaders:
    print(f"{team_leader.position} team:")
    for worker in team_leader.workforce:
        print(f"Position: {worker.position}, Core Principles: \n{worker.core_principles}")
    print("----")

Teams and members:
Lead Prompt Writer team:
Position: Style and Structure Expert, Core Principles: 
- Always structure prompts logically, for example COSTAR (Context, Objective, Steps, Task, Additional Information, Result)
- Always use a style and tone in prompts that is appropriate for the task
- Always consider the complexity of the task when designing prompts
Position: Conciseness and Clarity Expert, Core Principles: 
- Always write clear and concise prompts
- Always use simple and direct language in prompts
- Always consider the task criteria when formatting and structuring prompts
Position: Contextual Relevance Expert, Core Principles: 
- Always provide context to help the model understand the task
- Always consider the context in which prompts will be used
- Always ensure personas and roles in prompts are optimal to the context of the task
Position: Task Alignment Expert, Core Principles: 
- Always ensure that prompts align with the task criteria
- Always tailor instructions to t

In [67]:
result = leader_agent.optimise_prompt()

Approval results: [False, False]
================================ Human Message =================================
Name: Lead Prompt Writer

Feedback: To improve the prompt, explicitly state the specific maths problem to be solved to avoid ambiguity. Rephrase the instructions to ensure clarity by stating: 'Solve the math problem and display the answer as ##<your answer> with no spaces or units at the end.' Emphasize the importance of solving the problem first, then formatting the answer correctly. Additionally, remind the LLM to double-check the answer format to maintain consistency.
================================== Ai Message ==================================
Name: Leader

Updated Prompt: Solve the math problem and display the answer as ##<your answer> with no spaces or units at the end. Please double-check the answer format to maintain consistency. Your task is to make updates to the prompt above by implementing the feedback below. You must action the feedback keeping in mind your 

In [68]:
result["prompt"]

'Solve the math problem and display the answer as ##<answer> with no spaces or units at the end. Use Arabic numerals. For example, for 2+2, the output should be ##4. Double-check the answer format to maintain consistency.'